# NLP Assignment 1

Submitted by: <span style="color:red"> INSERT YOUR NAME HERE </span>


<hr>

## Assignment Goals
The goal of this assignment is to familiarize yourself with:
1. Regular expressions,
2. Text normalization,
3. Edit distance,
4. N-gram language models and smoothing

This assignment combines tutorial components, with learning exercises that you must complete and submit. The learning exercise sections are clearly demarcated within the assignments.



<hr>

# Part 0: Collecting Data


Before we can perform any text analysis, we will need to collect some text to apply our analysis to! One excellent resource for free text data is [Project Gutenburg](https://www.gutenberg.org/) (PG); PG contains ~60,000 free eBooks in a variety of formats including the very easy to use `.txt` format. Let's start by pulling [Bertrand Russell's](https://plato.stanford.edu/entries/russell/) book on [The Problems of Philosophy](http://www.gutenberg.org/cache/epub/5827/pg5827.txt) directly from the internet to our machines using [Python's requests library](https://requests.readthedocs.io/en/master/). The requests library is a powerful tool for extracting text data from the internet. Given that most contemporary text data is on the web, it's a tool you should become familiar with:

In [1]:
import requests
website = 'http://www.gutenberg.org/cache/epub/5827/pg5827.txt'
site    = requests.get(website)

In [2]:
print('\n')
print('--------------------------------------------------------')
print('Server status code: ' + str(site.status_code))
print('--------------------------------------------------------')
print('Here is a 250 character sample of your text:')
print('- - - - - - - - - - - - - - - - - - - - - - - - - - - - ')
print(site.text[1500:1750])
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Server status code: 200
--------------------------------------------------------
Here is a 250 character sample of your text:
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
ds the relations
of sense-data to physical objects, and from the latter as regards
probability and induction. I have also profited greatly by the
criticisms and suggestions of Professor Gilbert Murray.

1912




CHAPTER I. APPEARANCE AND RE
--------------------------------------------------------




The requests object returned a server status code of `200`, and placed the text we requested in `r.text`. The status code is the server's way of telling us `OK` but note that servers can also refuse to serve us; If we request another resources that doesn't exist from Guttenburg for example, it will respond differently. Let's ask for a new book called `idontthinkyouhavethisbook.txt`

In [3]:
website = 'http://www.gutenberg.org/cache/epub/5827/idontthinkyouhavethisbook.txt'
r = requests.get(website)

In [4]:
print('\n')
print('--------------------------------------------------------')
print('Server status code: ' + str(r.status_code))
print('--------------------------------------------------------')
print('Here is a 250 character sample of your text:')
print('- - - - - - - - - - - - - - - - - - - - - - - - - - - - ')
print(r.text[1500:1750])
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Server status code: 404
--------------------------------------------------------
Here is a 250 character sample of your text:
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
_logo"> -->
  <a id="main_logo" href="/" class="no-hover">
    <img src="/gutenberg/pg-logo-129x80.png" alt="Project Gutenberg" draggable="false" />
  </a>
  <!--	</div>-->
  <div id="menu">
    <label for="tm" id="toggle-menu">Menu<span class="drop-
--------------------------------------------------------




Now the server returned a `404` status code, which is it's way of telling us `Not Found`. Sure enough, the content that was returned in `r.text` is the error page of Project Guttenburg, not the book we requested. I point this out because when you collect your own data from the internet, especially large volumes of data, you will want to pay attention to those server status codes to make sure what you requested is what you obtained. To learn more about server codes, you can see this [this page](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

<hr>

# Part 1: Regular Expressions

A regular expression is a compact programming language that is specialized for enhanced text search. We can import the `re` library, which comes standard with the Python programming language, to analyze the book we pulled from Project Guttenburg in part 0.

In [5]:
import re

In [6]:
regular_expression = 'philosophy'
text               = site.text
results            = re.search(regular_expression,text)

In [7]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('A match for the search query: "' + results.group() + '"\nwas found at the following character span: ' + str(results.span()))
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
A match for the search query: "philosophy"
was found at the following character span: (1039, 1049)
--------------------------------------------------------




The above example finds the first instance of the word `philosophy` in the book. For some purposes, we may want to capture *every instance* where the search term was mentioned in a text. We can accomplish that using the `re.finditer` function, and packaging our results in a neat list of dictionaries that contain the matching string and the location of the match:

In [8]:
# collect all the mentions of the search term
regular_expression = 'philosophy'
results            = [ {"indicies":m.span(), "match":m.group() } for m in re.finditer(regular_expression,text)];
matches            = []; [matches.append(m['match']) for m in results];

In [9]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('We found ' +  str(len(matches)) + ' matches...\n')
print(str(results))
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
We found 72 matches...

[{'indicies': (1039, 1049), 'match': 'philosophy'}, {'indicies': (2108, 2118), 'match': 'philosophy'}, {'indicies': (2124, 2134), 'match': 'philosophy'}, {'indicies': (5384, 5394), 'match': 'philosophy'}, {'indicies': (20166, 20176), 'match': 'philosophy'}, {'indicies': (21445, 21455), 'match': 'philosophy'}, {'indicies': (33758, 33768), 'match': 'philosophy'}, {'indicies': (33835, 33845), 'match': 'philosophy'}, {'indicies': (34124, 34134), 'match': 'philosophy'}, {'indicies': (50596, 50606), 'match': 'philosophy'}, {'indicies': (55035, 55045), 'match': 'philosophy'}, {'indicies': (103572, 103582), 'match': 'philosophy'}, {'indicies': (109393, 109403), 'match': 'philosophy'}, {'indicies': (122675, 122685), 'match': 'philosophy'}, {'indicies': (122804, 122814), 'match': 'philosophy'}, {'indicies': (126227, 126237), 'match': 'philosophy'}, {

We found `72` matches for the search term. That seems a bit low for a book with the word philosophy in it's name! Let's update our regular expression so that it's not sensitive to capitalization of the letter `p`.

In [10]:
# collect all the mentions of the search term
regular_expression = '(p|P)hilosophy'
results            = [ {"indicies":m.span(), "match":m.group() } for m in re.finditer(regular_expression,text)];
matches            = []; [matches.append(m['match']) for m in results];

In [11]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('We found ' +  str(len(results)) + ' matches...\n')
print(str(matches))
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
We found 81 matches...

['Philosophy', 'Philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'Philosophy', 'philosophy', 'philosophy', 'Philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'Philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy', 'Philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy

We found `81` matches; that is not much better than before. Let's make the search even more general by seeking out terms that start with the word `philo`, followed by an arbitrary number of [a-z] characters thereafter.

In [12]:
regular_expression = '(P|p)hilo[a-z]+'
results            = [ {"indicies":m.span(), "match":m.group() } for m in re.finditer(regular_expression,text)];
matches            = []; [matches.append(m['match']) for m in results];

In [14]:
def CountFrequency(my_list):

    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return freq

In [15]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('We found ' +  str(len(matches)) + ' matches...\n')
print(str(matches))
print('--------------------------------------------------------')
print('We found ' +  str(len(set(matches))) + ' distinct terms...\n')
print(CountFrequency(matches))
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
We found 154 matches...

['Philosophy', 'Philosophy', 'philosophy', 'philosophers', 'philosophy', 'philosophy', 'philosophy', 'philosopher', 'philosopher', 'philosopher', 'Philonous', 'Philonous', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'Philosophy', 'philosophy', 'philosophy', 'philosopher', 'philosophical', 'Philosophy', 'philosophy', 'philosophers', 'philosophy', 'philosophy', 'philosophical', 'philosopher', 'philosophical', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophy', 'philosophical', 'philosopher', 'philosophy', 'philosopher', 'philosophy', 'philosopher', 'philosophy', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosopher', 'philosophers', 'philosophy', 'philosophy', 'philosophical', 'philosophers', 'philosophy', 'philosophy', 'philosophers', 'philo

That looks better! But now we have a new problem. It seems that an inconveniently named philosopher, `Philonous`, has made his way into our results. Let's modify our regular expression to use a `negative lookahead` `(?!nous)` to remove him.

In [16]:
regular_expression = '(P|p)hilo(?!nous)[a-z]+'
results            = [ {"indicies":m.span(), "match":m.group() } for m in re.finditer(regular_expression,text)];
matches            = []; [matches.append(m['match']) for m in results];

In [18]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('We found ' +  str(len(matches)) + ' matches...\n')
print(str(matches))
print('--------------------------------------------------------')
print('We found ' +  str(len(set(matches))) + ' distinct terms...\n')
print(CountFrequency(matches))
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
We found 151 matches...

['Philosophy', 'Philosophy', 'philosophy', 'philosophers', 'philosophy', 'philosophy', 'philosophy', 'philosopher', 'philosopher', 'philosopher', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'Philosophy', 'philosophy', 'philosophy', 'philosopher', 'philosophical', 'Philosophy', 'philosophy', 'philosophers', 'philosophy', 'philosophy', 'philosophical', 'philosopher', 'philosophical', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosophy', 'philosophical', 'philosopher', 'philosophy', 'philosopher', 'philosophy', 'philosopher', 'philosophy', 'philosophers', 'philosophers', 'philosophers', 'philosophers', 'philosopher', 'philosophers', 'philosophy', 'philosophy', 'philosophical', 'philosophers', 'philosophy', 'philosophy', 'philosophers', 'philosophy', 'philosophers', 'p

<hr>

## Learning Exercise 1:
#### Worth 1/5 Points
I'm curious how Bertrand used these various philosophical terms in his sentences, and how Bertrand's language, more generally, compares to [Friedrich Nietzsche](https://en.wikipedia.org/wiki/Friedrich_Nietzsche) and other philosophers.

#### A. Download Data
Use Python's `requests` library to collect Friedrich Nietzsche's book, [Beyond Good and Evil](http://www.gutenberg.org/cache/epub/4363/pg4363.txt) into Python. Collect one additional work of philosophy (of your choice) and import it as well.

In [31]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.

import requests
website = 'https://www.gutenberg.org/cache/epub/4363/pg4363.txt'
site    = requests.get(website)

################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

In [32]:
print('\n')
print('--------------------------------------------------------')
print('Server status code: ' + str(site.status_code))
print('--------------------------------------------------------')
print('Here is a 250 character sample of your text:')
print('- - - - - - - - - - - - - - - - - - - - - - - - - - - - ')
print(site.text[1500:1750])
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Server status code: 200
--------------------------------------------------------
Here is a 250 character sample of your text:
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
for most foreign
language phrases that were italicized. Original footnotes are put in
brackets "[]" at the points where they are cited in the text. Some
spellings were altered. "To-day" and "To-morrow" are spelled "today"
and "tomorrow." Some wor
--------------------------------------------------------




#### B. Basic Text Cleansing
Use `re.sub` to do some basic text cleansing on books you downloaded. More specifically:

1. replace the various forms of whitespace that appear (`\n`,`\r`,`\t` etc.) with a single whitespace character.
2. cast all text to lower case
3. replace common abbreviations with their full form, for instance: `it's` with `it is`

Note that item #3 does not have to be comprehensive, just do a couple common abbreviations to illustrate that you know how to do this.

In [33]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.

################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

#### C. Sentence Extractor
Write a regular expression that will extract the complete sentences containing any matches of the following regex we wrote together earlier: `(P|p)hilo(?!nous)[a-z]+`. Apply this regular expression to "Beyond Good and Evil", "The Problems of Philosophy", and the work of philosophy you chose. Show a couple of example sentences from each work.

In [1]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR RESULTS TO THE SCREEN.
# collect all the mentions of the search term
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

#### D. Sentence Tokenizer
Write another regular expression that tokenizes the text in your collected sentences by splitting the sentences into a list of individual words. Tokenize the entire text of all three books using this regular expression. For each book, store your result as a list of lists and print the last word of the last sentence.

In [ ]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR RESULTS TO THE SCREEN.
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

#### E. Compare Works
Use Python's `matplotlib` library, to compare the distribution of the most common words that show up in each book. Compare the distributions and comment on any important statistical similarities or differences between the distributions. Do the distributions of the word frequency follow Zipf's law? Ignoring common words (`the`,`and`,`in`, etc.) what are the top 10 words that each book used most frequently? Are there any words that showed up more frequently in one work than they did in another?

In [2]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

<hr>

# Part 2: Text normalization
Text normalization is an essential preliminary step to any natural language processing pipeline. To begin, let's re-import `The Problems of Philosophy`

In [41]:
website  = 'http://www.gutenberg.org/cache/epub/5827/pg5827.txt'
text     = requests.get(website).text

### Tokenization
As you saw in `Learning Exercise 1`, tokenization of text can be performed through the use of Regular Expressions. Familiarity with Regular Expressions is an important part of any NLP apprentice's toolkit but fortunately, it's not the only (or even the best) tool that exists for text normalization tasks! Indeed, for basic text normalization and cleansing, Python has a number of functions that operate on strings directly - no libraries required!

Let's begin this section of the tutorial by taking a look at some of these functions, starting with the `replace` function to remove the whitespace characters in our text data:

In [42]:
#Replace \r\n with whitespace, then replace \n with whitespace, then replace \r with whitespace
clean_text = text.replace('\r\n',' ').replace('\n',' ').replace('\r',' ')

In [43]:
print('\n')
print('--------------------------------------------------------')
print('Results')
print('--------------------------------------------------------')
print('Before:\n' + text[1101:1227] + ' ...')
print('--------------------------------------------------------')
print('After:\n' + clean_text[1056:1177] + ' ...')
print('--------------------------------------------------------')
print('\n')



--------------------------------------------------------
Results
--------------------------------------------------------
Before:
mething positive and constructive, since merely negative criticism
seemed out of place. For this reason, theory of knowledge  ...
--------------------------------------------------------
After:
hing positive and constructive, since merely negative criticism seemed out of place. For this reason, theory of knowledge ...
--------------------------------------------------------




That seems to have worked! Notice that `replace` is a method that is built into Python strings. That's the reason that we write `text.replace(...)` and not something like `replace(text,...)`.  

Python has other many [other useful string manipulation methods](https://www.programiz.com/python-programming/methods/string), but for the rest of the basic pre-processing we'll be used in this tutorial, we'll only need `split`:

In [44]:
# Split the sentences when we see a '.' character
clean_text = clean_text.split('.')

# Split each sentence further - when there are no arguments provided to split, it defaults to whitespace (tabs, newlines, spaces, etc)
textmat = []
for c in clean_text:
    textmat.append(c.split())
#re.split(r'\w+|\$[\d\.]+|\S+',"text")

In [47]:
# Printing the tenth sentence:
print('--------------------------------------------------------')
print('Results (showing sentence 10)')
print('--------------------------------------------------------')
print(textmat[5])
print('--------------------------------------------------------')
print('\n')

--------------------------------------------------------
Results (showing sentence 10)
--------------------------------------------------------
['Title:', 'The', 'Problems', 'of', 'Philosophy', 'Author:', 'Bertrand', 'Russell', 'Release', 'date:', 'June', '1,', '2004', '[eBook', '#5827]', 'Most', 'recently', 'updated:', 'December', '5,', '2019', 'Language:', 'English', 'Credits:', 'Produced', 'by', 'Gordon', 'Keener,', 'and', 'David', 'Widger', '***', 'START', 'OF', 'THE', 'PROJECT', 'GUTENBERG', 'EBOOK', 'THE', 'PROBLEMS', 'OF', 'PHILOSOPHY', '***', 'Produced', 'by', 'Gordon', 'Keener', 'THE', 'PROBLEMS', 'OF', 'PHILOSOPHY', 'By', 'Bertrand', 'Russell', 'PREFACE', 'In', 'the', 'following', 'pages', 'I', 'have', 'confined', 'myself', 'in', 'the', 'main', 'to', 'those', 'problems', 'of', 'philosophy', 'in', 'regard', 'to', 'which', 'I', 'thought', 'it', 'possible', 'to', 'say', 'something', 'positive', 'and', 'constructive,', 'since', 'merely', 'negative', 'criticism', 'seemed', 'out', 

The pre-processing shown above is mostly for illustrative purposes. If we'd like to do anything serious, we'll need a more sophisticated approach to sentence segmentation and text tokenization. Fortunately, there are many existing tools in Python that assist with the Sentence segmentation and word tokenization. One of the most popular tools for this is the [Natural Langauge Toolkit](https://www.nltk.org/). Let's import the tool into python and explore some of it's functionality together:

In [48]:
# Importing the NLTK Package
import nltk

# Importing the `punkt` data, used by the NLTK tokenizer
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Now, let's test out the NLTK tokenizer with a tricky sentence

In [49]:
sentences = """At eight o'clock on Thursday morning Arthur didn't feel very good. He drove to the store in New York at 100 m.p.h.! He wanted to obtain some Kool-Aid! It cost $10.89."""
tokens    = nltk.word_tokenize(sentences)
print('\n')
print(tokens)
print('\n')



['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', 'Arthur', 'did', "n't", 'feel', 'very', 'good', '.', 'He', 'drove', 'to', 'the', 'store', 'in', 'New', 'York', 'at', '100', 'm.p.h.', '!', 'He', 'wanted', 'to', 'obtain', 'some', 'Kool-Aid', '!', 'It', 'cost', '$', '10.89', '.']




As you can see, the tokenization is pretty good for something out-of-the box, but it could be better! In an ideal world, we would want the Tokenizer to understand that `New York` is actually a single token, not two. Furthermore, depending on our use case, we may want the `$` sign to be attached to the `10.89` token. So, if you wanted to use the NLTK tool out-of-the-box, you may need to do some additional pre-processing. Let's try the sentence splitting functionality next:

In [50]:
result = nltk.sent_tokenize(sentences)
print('\n' + str(result) + '\n')


["At eight o'clock on Thursday morning Arthur didn't feel very good.", 'He drove to the store in New York at 100 m.p.h.!', 'He wanted to obtain some Kool-Aid!', 'It cost $10.89.']



This one also works rather nicely. Indeed, NLTK does a lot to make Natural Language Processing accessible.

<hr>

## Learning Exercise 2:
#### Worth 1/5 Points

For this learning exercise, you will implement the Byte Pair Encoding (BPE) algorithm and use it to Tokenize `The Problems of Philosophy`. If you can't recall how the BPE algorithm works, you can refer back to the lectures, or see Section 2.4.3 of [Daniel Jurafsky & James H. Martin's NLP book](https://web.stanford.edu/~jurafsky/slp3/2.pdf).

#### A. Download Books to Create a Larger Corpus
Given that the BPE algorithm is data-driven, we'll first need to gather a training data corpus. For your training data, you will choose at least 10 books from [Project Gutenburg](https://www.gutenberg.org/):
* Choose at least 5 books that were authored by Bertrand Russell, not including `The Problems of Philosophy`
* Choose at least 5 books that were authored by Friedrich Nietzsche

You'll be using these training data later with BPE to identify the token `vocabularies`, which you will then use to tokenize `The problems of Philosophy`.

In [ ]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

#### B. Implement Byte Pair Encoding
Implement the Byte Pair Encoding algorithm by filling out the function skeleton below. The function should take three inputs:

* `text` : an unprocessed text file that we wish to tokenize using BPE
* `k`    : the number of merges to perform
* `training_corpus` : a python list of complete texts we wish to use for training. The formatting should look like: `['blah blah blah ...', 'the rain in spain ...', 'do pickles count as vegetables? ...']`

The function should output a tokenized version of the `text` based on the `k` vocabulary units learned by BPE using the `training_corpus`.

In [ ]:
def BytePairEncoding(text, k, training_corpus):
    tokenized_text = []
    vocabulary = []
    # INSERT YOUR CODE HERE
    return vocabulary, tokenized_text

#### C. Evaluate Byte Pair Encoding
Because BPE is data-driven, we might expect there to be some differences in the tokenization based on the training corpus, as well as your choice of `k`! To evaluate BPE, you will now experiment with how the tokenization performs for various settings of `k`, and when using different training . More specifically, you will compare the `vocabulary` and `tokenized_text` of Bertrand Russell's `The Problem's of Philosophy` for:

* values of `k` ranging from 1,000 to 10,000 in steps of 1,000 and
    *  `training_corpus` sets that include: (1) only Bertrand Russell books, (2) only Friedrich Nietzsche books, and (3) all books from groups (1) and (2).

For the above conditions, please generate one or more plots using Python's `matplotlib` that compare:
* The total number of unique tokens in the tokenized text
* The median size of tokens in the tokenized text
* The % overlap in the number of distinct unique tokens generated by BPE and NLTK's word tokenizer.

Compare the plots, and note any interesting observations about the results.

In [ ]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

<hr>

# Part 3: Edit Distance
#### Character Level Distance
As we discussed in the lectures, Edit Distance is a way to measure the similarity between different sequences of text. Our goal in this section will be to compare and contrast some of the different measures of edit distance that exist so that you'll have a better intuition for how to use them, depending on the problem you're trying to solve. To start, let's import the [`textdistance` library](https://github.com/life4/textdistance), written by [Gram Voronov](https://orsinium.dev/).

In [5]:
!pip install textdistance
import textdistance

The `textdistance` library contains python implementations of about 30 edit distance algorithms. Let's begin by trying out the edit distance tool we discussed in class, the [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance):

In [6]:
def GetDistance(string1, string2, method):
    # Compute the distance and print the results
    distance = eval("textdistance." + method + "(string1, string2)")
    print('"' + string1 + '" and "' + string2 + '" is: ' + str(distance))

print('\n')
print('levenshtein' + " distance:")
print('--------------------------------------------------------')
GetDistance('test','test'      ,'levenshtein')
GetDistance('test','texts'     ,'levenshtein')
GetDistance('test','   test'   ,'levenshtein')
print('--------------------------------------------------------')
print('\n')




levenshtein distance:
--------------------------------------------------------
"test" and "test" is: 0
"test" and "texts" is: 2
"test" and "   test" is: 3
--------------------------------------------------------




Recall from the lecture that the Levenshtein distance counts the total number of `insertion`, `deletion`, and `substitution` actions that a required to transform one string into another and the cost of each edit is always 1. So, as shown in the above example, moving from `"test"` to `"test"` requires no edits, and therefore has a distance of 0, moving from `"test"` to `"texts"` requires 1 substitution and one addition and therefore has a distance of 2, moving from `"test"` to `"   test"`   has a distance of 3 becasue we must remove the  three whitespace characters to the left of the word `"   test"` to create a match.

The above example is intentionally meant to highlight that the assumptions of the edit distance measures can have consequence for their estimates; Indeed, depending on the problem we are trying to solve, we may want `"   test"` and `"test"` to be closer than the distance between `"test"` and `"texts"`. On that note, let's explore the performance of three other edit distance measures: the [Hamming Distance](https://en.wikipedia.org/wiki/Hamming_distance), [Needleman Wunsch](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm), and [Smith Waterman](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm):

In [7]:
distance_measures = ['hamming',  'levenshtein', 'needleman_wunsch', 'smith_waterman']

for method in distance_measures:
    print(method + " distance:")
    print('--------------------------------------------------------')
    GetDistance('test'      ,'test'      ,method)
    GetDistance('test'      ,'texts'     ,method)
    GetDistance('test'      ,'   test'   ,method)
    print('--------------------------------------------------------')
    print('\n')



hamming distance:
--------------------------------------------------------
"test" and "test" is: 0
"test" and "texts" is: 2
"test" and "   test" is: 6
--------------------------------------------------------


levenshtein distance:
--------------------------------------------------------
"test" and "test" is: 0
"test" and "texts" is: 2
"test" and "   test" is: 3
--------------------------------------------------------


needleman_wunsch distance:
--------------------------------------------------------
"test" and "test" is: 4.0
"test" and "texts" is: 2.0
"test" and "   test" is: 1.0
--------------------------------------------------------


smith_waterman distance:
--------------------------------------------------------
"test" and "test" is: 4
"test" and "texts" is: 2.0
"test" and "   test" is: 4.0
--------------------------------------------------------




Let's step through these results together. First, we notice that the Hamming distance between `"test"` and `"   test"` is 6, which is even higher than what we observed for the Levenshtein distance! To understand why, we can refer to the definition of the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance): "The Hamming distance between two strings is the number of positions at which the corresponding symbols are different. In other words, it measures the minimum number of substitutions required to change one string into the other." Given that there are so many white space characters, and that Hamming is only concerned with substitutions, it makes sense that the hamming distance would be higher than the levenshtein distance!

At first glance, the results of the [Needleman Wunch](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) and [Smith Waterman](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm) methods seem suprising. Why is the distance between "test" and "test" 4? Shouldn't it be 0? Once again, the answer here resides in understanding the definition of the "distance" metric. Both of these algorithms are actually providing an **alignment score** between the two sequences.

Of course, we may also notice that the scores between the two alignment methods are quite different; the reason for this difference has to do with what these alignment scores are exactly measuring: the Smith–Waterman algorithm is designed to find the *segments* in two sequences that have maximal similarities while the Needleman–Wunsch algorithm is designed for alignment between two complete sequences. Let's explore these two methods to make this distinction a little more clear:

In [8]:
distance_measures = ['needleman_wunsch', 'smith_waterman']
for method in distance_measures:
    print(method + " distance:")
    print('--------------------------------------------------------')
    GetDistance('test','test',method)
    GetDistance('test',' test',method)
    GetDistance('test','  test',method)
    GetDistance('test','   test',method)
    GetDistance('test','    test',method)
    GetDistance('test','     test',method)
    GetDistance('test','      test',method)
    GetDistance('test','       test',method)
    print('\n')


needleman_wunsch distance:
--------------------------------------------------------
"test" and "test" is: 4.0
"test" and " test" is: 3.0
"test" and "  test" is: 2.0
"test" and "   test" is: 1.0
"test" and "    test" is: 0.0
"test" and "     test" is: -1.0
"test" and "      test" is: -2.0
"test" and "       test" is: -3.0


smith_waterman distance:
--------------------------------------------------------
"test" and "test" is: 4
"test" and " test" is: 4.0
"test" and "  test" is: 4.0
"test" and "   test" is: 4.0
"test" and "    test" is: 4.0
"test" and "     test" is: 4.0
"test" and "      test" is: 4.0
"test" and "       test" is: 4.0




As we can see above, the Needleman Wunsh method grows more negative as the absolute difference between the two strings increases. In this sense, the Needleman-Wunsch algorithm is similar to the Levenshtein distance algorithm; the difference being that Levenshtein uses a static penalty cost to any mismatched letters while the Needleman-Wunsch algorithm gives weights to matches and mismatches differently. The Smith Waterman method distance remains 4 because the substring `test` overlaps in both strings regardless of the whitespace.

#### Token-Level distance
All of our edit distance work up until now has focused on character-level distances and alignment but depending on the problem we want to solve, we may care more about distance at the token level. Fortunately, there are several distance metrics that measure similarity between strings at the token level. One such measure is the [Jaccard Index](https://en.wikipedia.org/wiki/Jaccard_index), which is the size of the intersection of the token sets divided by the size of the union of the sample sets. Here's an example:

In [9]:
distance_measures = ['jaccard']
text1 = "The fish was delish, and it made quite a dish"
text2 = "The fish on the dish was made quite delicious!"
text3 = "Joe eats crabs, crabs do not taste good."
for method in distance_measures:
    print(method + " distance:")
    print('--------------------------------------------------------')
    GetDistance(text1      ,text2      ,method)
    GetDistance(text1      ,text3      ,method)
    print('--------------------------------------------------------')
    print('\n')

jaccard distance:
--------------------------------------------------------
"The fish was delish, and it made quite a dish" and "The fish on the dish was made quite delicious!" is: 0.7843137254901961
"The fish was delish, and it made quite a dish" and "Joe eats crabs, crabs do not taste good." is: 0.39344262295081966
--------------------------------------------------------




We won't be covering all the edit distance functions of the NLP community in great detail, but hopefully this is enough to whet your appetite, and allow you to dig into some of the methods on your own.

<hr>

## Learning Exercise 3:
#### Worth 1/5 Points
#### A. Towards a More Nuanced Edit Distance Cost Metric
As we've eluded to here and in the lectures, one deficiency of the Levenshtein cost metric is the assumption that substitutions, edit, and deletion costs are all equal. Your goal for this learning exercise is to modify the cost structure of the `LevenshteinDistanceDP` method shown below so that is assigns 1/2 of the normal penalty for any edits that involve whitespace characters (FYI, the implementation shown below was originally written by [Ahmed Gad](https://blog.paperspace.com/author/ahmed/), [here](https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/#:~:text=The%20Levenshtein%20distance%20is%20a,representing%20the%20distance%20between%20them.&text=In%20this%20tutorial%20the%20Levenshtein,using%20the%20dynamic%20programming%20approach.)).

In [10]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.
################################################################################

import numpy as np
def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2

    a = 0
    b = 0
    c = 0

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]

                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

#

levenshteinDistanceDP('test  ', 'test')

2.0

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

<hr>

# Part 4: N-Gram language Models
In this section, we'll be covering n-gram language models which, as the name implies, are language models that use n-grams!

#### N-grams
An n-gram is a contiguous sequence of n items from a given sample of text or speech. Let's begin this section by defining a function that generates n-grams from our text data using the `nltk` library and using it to extract n-grams for n ranging from 1-5.

In [12]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams

# an ngram extractor
def extract_word_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

# some example text
data = """a fish keeps for a day; a fish keeps well if you put it in a cold place.
          a fish keeps best if you put it in the fridge. If you’ll be having fish,
          have it with an apple because one apple a day keeps the doctor away!
          You know they also say that a day keeps coming."""

# printing the first 5 tokens after gramification
print("\nWord-Level:")
print("1-gram: ", extract_word_ngrams(data, 1)[0:5])
print("2-gram: ", extract_word_ngrams(data, 2)[0:5])
print("3-gram: ", extract_word_ngrams(data, 3)[0:5])
print("4-gram: ", extract_word_ngrams(data, 4)[0:5])
print("\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



Word-Level:
1-gram:  ['a', 'fish', 'keeps', 'for', 'a']
2-gram:  ['a fish', 'fish keeps', 'keeps for', 'for a', 'a day']
3-gram:  ['a fish keeps', 'fish keeps for', 'keeps for a', 'for a day', 'a day ;']
4-gram:  ['a fish keeps for', 'fish keeps for a', 'keeps for a day', 'for a day ;', 'a day ; a']




Notice that the n-gram extractor I use above leverages the `nltk` tokenizer that we discussed earlier in this assignment. Also, that sample data I showed above should look familiar... As we discussed in the lectures, n-grams can be generated at whatever level of resolution we need for the task at hand; we also don't need any fancy libraries to generate them! If we were interested in extracting n-grams at the character level for instance, we could do that in one line:

In [13]:
# some example text
data = """a fish keeps for a day; a fish keeps well if you put it in a cold place.
          a fish keeps best if you put it in the fridge. If you’ll be having fish,
          have it with an apple because one apple a day keeps the doctor away!
          You know they also say that a day keeps coming."""

def extract_char_ngrams(data, num):
    return [data[i:i+num] for i in range(len(data))][:(-num-1)]

# printing the first 5 tokens after gramification
print("\nCharacter-Level:")
print("1-gram: ", extract_char_ngrams(data, 1)[0:5])
print("2-gram: ", extract_char_ngrams(data, 2)[0:5])
print("3-gram: ", extract_char_ngrams(data, 3)[0:5])
print("4-gram: ", extract_char_ngrams(data, 4)[0:5])
print("\n")


Character-Level:
1-gram:  ['a', ' ', 'f', 'i', 's']
2-gram:  ['a ', ' f', 'fi', 'is', 'sh']
3-gram:  ['a f', ' fi', 'fis', 'ish', 'sh ']
4-gram:  ['a fi', ' fis', 'fish', 'ish ', 'sh k']




#### Skip-Grams
While n-grams bring together contiguous sequences of words, skip-grams include grams that skip over certain terms!

In [20]:
# some example text
data = """a fish keeps for a day; a fish keeps well if you put it in a cold place.
          a fish keeps best if you put it in the fridge. If you’ll be having fish,
          have it with an apple because one apple a day keeps the doctor away!
          You know they also say that a day keeps coming."""

# Creating skip-grams using the NLTK library
skip_gram = list(nltk.skipgrams(nltk.word_tokenize(data),n=2,k=2))
skip_gram[0:5]

[('a', 'fish'),
 ('a', 'keeps'),
 ('a', 'for'),
 ('fish', 'keeps'),
 ('fish', 'for')]

#### Language Models
A language model refers to any method that assign probabilities to strings. Let's try to develop a more concrete understanding of language models by building a simple one together. More specifically, let's build a model that tries to predict the next word in a sentence, given the last word we saw. One approach to accomplish this is to simply count, for each unique word, what words tend to follow it. To do that, let's start by using the `extract_word_ngrams` tool we showed earlier, and collect all the unique words in the corpus:

In [21]:
# some example text
data = """a fish keeps for a day; a fish keeps well if you put it in a cold place.
          a fish keeps best if you put it in the fridge. If you’ll be having fish,
          have it with an apple because one apple a day keeps the doctor away!
          You know they also say that a day keeps coming."""

unigrams = extract_word_ngrams(data,1)

# get the unique words:
vocabulary = list(set(unigrams))
print('---------------------------------------')
print('Vocabulary (n = ' + str(len(vocabulary)) +')')
print('---------------------------------------')
print(str(vocabulary) + '\n')

---------------------------------------
Vocabulary (n = 40)
---------------------------------------
['day', 'an', 'they', 'because', '.', 'doctor', '!', 'if', 'also', 'one', 'put', 'that', 'll', 'best', 'the', 'say', 'coming', 'keeps', 'in', 'cold', 'You', 'away', 'you', 'be', 'for', 'fridge', 'well', ';', '’', 'fish', 'with', ',', 'apple', 'it', 'place', 'having', 'have', 'a', 'know', 'If']



Now that we have a list of the words, lets initialize a `JSON` object called `counts` that will keep track of the next word, given the current word.

In [22]:
counts = {}
for given_word in vocabulary:
    counts[given_word] = {}
    for next_word in vocabulary:
        counts[given_word][next_word] = 0

The `counts` object is a dictionary of dictionaries. It's convenient to store the count data this way because we can see the counts of the next word, given the current word, by simply providing the count object the current word. Let's take a look at the count of all possible next words, given that the current word is `fish`:

In [23]:
print('------------------------------------------------------')
print('Count of next words, given that current word is "fish"')
print('------------------------------------------------------')
print(counts['fish'])
print('\n')

------------------------------------------------------
Count of next words, given that current word is "fish"
------------------------------------------------------
{'day': 0, 'an': 0, 'they': 0, 'because': 0, '.': 0, 'doctor': 0, '!': 0, 'if': 0, 'also': 0, 'one': 0, 'put': 0, 'that': 0, 'll': 0, 'best': 0, 'the': 0, 'say': 0, 'coming': 0, 'keeps': 0, 'in': 0, 'cold': 0, 'You': 0, 'away': 0, 'you': 0, 'be': 0, 'for': 0, 'fridge': 0, 'well': 0, ';': 0, '’': 0, 'fish': 0, 'with': 0, ',': 0, 'apple': 0, 'it': 0, 'place': 0, 'having': 0, 'have': 0, 'a': 0, 'know': 0, 'If': 0}




As we can see, the counts are currently empty; that's because we've only just initialized this object and haven't actually counted anything yet! Let's do that now.

In [24]:
for i in range(len(unigrams)-1):
    counts[unigrams[i]][unigrams[i+1]] += 1

Now let's take a look at that `fish` count again!

In [25]:
print('------------------------------------------------------')
print('Count of next words, given that current word is "fish"')
print('------------------------------------------------------')
print(counts['fish'])

print('\n')
print('------------------------------------------------------')
print('Count of next word "keeps", given that current word is "fish"')
print('------------------------------------------------------')
print(counts['fish']['keeps'])

print('\n')

------------------------------------------------------
Count of next words, given that current word is "fish"
------------------------------------------------------
{'day': 0, 'an': 0, 'they': 0, 'because': 0, '.': 0, 'doctor': 0, '!': 0, 'if': 0, 'also': 0, 'one': 0, 'put': 0, 'that': 0, 'll': 0, 'best': 0, 'the': 0, 'say': 0, 'coming': 0, 'keeps': 3, 'in': 0, 'cold': 0, 'You': 0, 'away': 0, 'you': 0, 'be': 0, 'for': 0, 'fridge': 0, 'well': 0, ';': 0, '’': 0, 'fish': 0, 'with': 0, ',': 1, 'apple': 0, 'it': 0, 'place': 0, 'having': 0, 'have': 0, 'a': 0, 'know': 0, 'If': 0}


------------------------------------------------------
Count of next word "keeps", given that current word is "fish"
------------------------------------------------------
3




Now that we have a count of the words, we can convert these to probabilities by simply dividing by the total incidence of next words, for a given current word:

In [26]:
# initialize the probabilites JSON object
probs = {}
for given_word in vocabulary:
    probs[given_word] = {}
    for next_word in vocabulary:
        probs[given_word][next_word] = 0

# convert the counts to probabilites
for key, value in counts.items():
    denominator = 0
    for key2, value2 in counts[key].items():
        denominator += value2

    for key2, value2 in counts[key].items():
        probs[key][key2] = value2 / denominator


Now, let's check the probability of the next word being `keeps`, given that the current word is `fish`

In [27]:
print('------------------------------------------------------')
print('The probability of the next word being keeps,')
print('given that the current word is fish:')
print('------------------------------------------------------')
print(str(100*probs['fish']['keeps']) + '%')
print('\n')

------------------------------------------------------
The probability of the next word being keeps,
given that the current word is fish:
------------------------------------------------------
75.0%




And there you have it! You have built your first data-driven language model. Congratulations! Now let's get to work doing some fun things with the language model, like simulating new sentences. Let's do that by choosing a word, and then selecting the next word in accordance to the probability given by our language model:

In [28]:
import numpy as np

def sample_next_gram_from_language_model(probs, given_token):
    distribution            = list(probs[given_token].values())
    sample_from_multinomial = np.random.multinomial(1,distribution)
    sample_index            = np.where(sample_from_multinomial==1)[0][0]
    word_keys               = list(probs[given_token].keys())
    next_word               = word_keys[sample_index]
    return(next_word)

given_token = "a"
next_token  = sample_next_gram_from_language_model(probs,given_token)

print(' Given the token :  ' + given_token)
print(' The next token is :  ' + next_token)
print('\n')


 Given the token :  a
 The next token is :  fish




Now that we have a language model built, we can do all sorts of useful things. Including running our model to generate new sentences we've never seen before:

In [30]:
def create_new_sentence(length, seed_token):
    tokens = [seed_token]
    for i in range(length):
        tokens.append(sample_next_gram_from_language_model(probs,tokens[-1]))
    return tokens

print('---------------------------------------')
print(' '.join(create_new_sentence(10, 'a')))
print(' '.join(create_new_sentence(10, 'a')))
print(' '.join(create_new_sentence(10, 'a')))

---------------------------------------
a day keeps best if you put it in the doctor
a day ; a fish keeps well if you ’ ll
a day keeps coming . a fish keeps well if you


<hr>

## Learning Exercise 4:
#### Worth 2/5 Points
Now that you have built your first language model, let's turn our attention back to the philosophers. For this final learning exercise, you will be building a tri-gram `Nietzsche language model`, and a tri-gram `Russel Language Model` using the 5 books from each of the philosophers you collected from Project Gutenburg earlier.

Generate a conversation between your philosophers models by seeding one of the models with a tri-gram and allow it to run until it generates a terminating character (a '.', '?' or '!'). Then, have the other model generate a response by providing the ending tri-gram of the the first model as the seed tri-gram for the other model. Iterate like this for 15 sentences and print the text.   

In [ ]:
################################################################################
# INSERT YOUR CODE HERE
# DO NOT FORGET TO PRINT YOUR MEANINGFUL RESULTS TO THE SCREEN.
################################################################################

<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>